In [109]:
import numpy as np
import os
import matplotlib.pyplot as plt 
import seaborn as sns
from keras.optimizers import Adam
import cv2
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout ,Conv2D
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras.utils import to_categorical
import pandas as pd
from keras.applications import VGG19,ResNet50
from keras.layers import Activation,BatchNormalization
from keras.layers import MaxPooling2D

In [110]:
train=pd.read_csv("/content/drive/MyDrive/Snake detection/Snake Detection/dataset/train.csv")
test=pd.read_csv("/content/drive/MyDrive/Snake detection/Snake Detection/dataset/test.csv")


In [111]:
test

,image_id
0,7ede553357
1,3500b219e4
2,d43a78d6d9
3,1f36f26994
4,8cb85fc58e
...,...
2356,93e591cb23
2357,68a603b39d
2358,4f472510f3
2359,d6884444ae


In [112]:
#img_width,img_heigth=32,32
train_data_dir='/content/drive/MyDrive/Snake detection/Snake Detection/dataset/train'
validation_data_dir='/content/drive/MyDrive/Snake detection/Snake Detection/dataset/test'



In [113]:
def append_ext(fn):
    return fn+".jpg"

train['image_id']=train['image_id'].apply(append_ext)

In [114]:
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,validation_split=0.2)
test_datagen=ImageDataGenerator(rescale=1./255,validation_split=0.2)

In [115]:
train_generator=train_datagen.flow_from_dataframe(
dataframe=train,
directory="/content/drive/MyDrive/Snake detection/Snake Detection/dataset/train",
x_col="image_id",
y_col="breed",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
subset="training",
target_size=(224,224))


validation_generator=train_datagen.flow_from_dataframe(
dataframe=train,
directory="/content/drive/MyDrive/Snake detection/Snake Detection/dataset/train",
x_col="image_id",
y_col="breed",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
subset="validation",
target_size=(224,224))

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 326 invalid image filename(s) in x_col="image_id". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 4146 validated image filenames belonging to 35 classes.
Found 1036 validated image filenames belonging to 35 classes.


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 326 invalid image filename(s) in x_col="image_id". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [116]:
IMG_SIZE=224

In [117]:
#base_model =  VGG19(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

base_model =  ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

In [118]:
model = Sequential()
model.add(Flatten(input_shape=base_model.output_shape[1:]))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(35))
model.add(Activation('softmax'))



model = Model(inputs=base_model.input, outputs=model(base_model.output))

#model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
 #             metrics=['accuracy'])
model.compile(loss='binary_crossentropy',
             optimizer='rmsprop'
            ,metrics=['accuracy'])

model.summary()


Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [119]:
add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dropout(0.3))
add_model.add(Dense(256, activation='relu'))
add_model.add(Dropout(0.3))
add_model.add(Dense(128, activation='relu'))
add_model.add(Dropout(0.3))
add_model.add(Dense(35, activation='softmax'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))

model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [127]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=4146//32,
                    validation_data=validation_generator,
                    validation_steps=1036//32,
                    epochs=10
                    )


Epoch 1/10
129/129 [==============================] - 77s 601ms/step - loss: 0.5376 - accuracy: 0.8388 - val_loss: 3.1646 - val_accuracy: 0.3193
Epoch 2/10
129/129 [==============================] - 77s 600ms/step - loss: 0.5351 - accuracy: 0.8352 - val_loss: 2.9309 - val_accuracy: 0.3271
Epoch 3/10
129/129 [==============================] - 78s 603ms/step - loss: 0.5363 - accuracy: 0.8369 - val_loss: 3.1324 - val_accuracy: 0.3135
Epoch 4/10
129/129 [==============================] - 80s 618ms/step - loss: 0.5194 - accuracy: 0.8452 - val_loss: 3.1958 - val_accuracy: 0.3232
Epoch 5/10
129/129 [==============================] - 80s 620ms/step - loss: 0.4598 - accuracy: 0.8641 - val_loss: 3.1638 - val_accuracy: 0.3164
Epoch 6/10
129/129 [==============================] - 80s 623ms/step - loss: 0.5198 - accuracy: 0.8381 - val_loss: 3.1792 - val_accuracy: 0.3057
Epoch 7/10
129/129 [==============================] - 81s 626ms/step - loss: 0.4526 - accuracy: 0.8551 - val_loss: 3.1685 - val_ac

In [ ]:
#Reused the model After 50 epoch , then again train . 

In [122]:
model.save("/content/drive/MyDrive/Snake detection/resnet50_snake_detection_2.hdf5")

In [125]:
from keras.models import load_model
model = load_model('/content/drive/MyDrive/Snake detection/resnet50_snake_detection_2.hdf5')

In [128]:
from tqdm import tqdm
test_img = []
for img_name in tqdm(test['image_id'].values):
    test_img.append(img_name + '.jpg')

100%|██████████| 2361/2361 [00:00<00:00, 827297.56it/s]


In [ ]:
test_img

In [130]:
TEST_PATH = '/content/drive/MyDrive/Snake detection/Snake Detection/dataset/test/'
test['image'] = test_img

test

,image_id,image
0,7ede553357,7ede553357.jpg
1,3500b219e4,3500b219e4.jpg
2,d43a78d6d9,d43a78d6d9.jpg
3,1f36f26994,1f36f26994.jpg
4,8cb85fc58e,8cb85fc58e.jpg
...,...,...
2356,93e591cb23,93e591cb23.jpg
2357,68a603b39d,68a603b39d.jpg
2358,4f472510f3,4f472510f3.jpg
2359,d6884444ae,d6884444ae.jpg


In [131]:
test_img = []
for img_name in tqdm(test['image'].values): 
    test_img.append(cv2.resize(cv2.imread(TEST_PATH + img_name),(224,224)))

100%|██████████| 2361/2361 [00:06<00:00, 339.06it/s]


In [132]:
len(test_img)

2361

In [133]:
test['image'][1]
t=cv2.imread(TEST_PATH + test['image'][1])
p=cv2.resize(t,(224,224))

In [135]:
u_test = np.array(test_img, np.float32) / 255


In [136]:
len(u_test)

2361

In [137]:
u_test.shape

(2361, 224, 224, 3)

In [138]:
#p_test = np.array(p, np.float32) / 255
#i_test=np.expand_dims(u_test,axis=0)

In [139]:
predictions = model.predict(u_test)

In [141]:
predictions = np.argmax(predictions, axis=1)

In [142]:
predictions

array([ 0,  1, 23, ..., 25, 23, 23])

In [143]:
len(predictions)

2361

In [144]:
dit=train_generator.class_indices
dit

{'agkistrodon-contortrix': 0,
 'agkistrodon-piscivorus': 1,
 'coluber-constrictor': 2,
 'crotalus-atrox': 3,
 'crotalus-horridus': 4,
 'crotalus-ruber': 5,
 'crotalus-scutulatus': 6,
 'crotalus-viridis': 7,
 'diadophis-punctatus': 8,
 'haldea-striatula': 9,
 'heterodon-platirhinos': 10,
 'lampropeltis-californiae': 11,
 'lampropeltis-triangulum': 12,
 'masticophis-flagellum': 13,
 'natrix-natrix': 14,
 'nerodia-erythrogaster': 15,
 'nerodia-fasciata': 16,
 'nerodia-rhombifer': 17,
 'nerodia-sipedon': 18,
 'opheodrys-aestivus': 19,
 'pantherophis-alleghaniensis': 20,
 'pantherophis-emoryi': 21,
 'pantherophis-guttatus': 22,
 'pantherophis-obsoletus': 23,
 'pantherophis-spiloides': 24,
 'pantherophis-vulpinus': 25,
 'pituophis-catenifer': 26,
 'rhinocheilus-lecontei': 27,
 'storeria-dekayi': 28,
 'storeria-occipitomaculata': 29,
 'thamnophis-elegans': 30,
 'thamnophis-marcianus': 31,
 'thamnophis-proximus': 32,
 'thamnophis-radix': 33,
 'thamnophis-sirtalis': 34}

In [150]:
result=[]
for i in predictions:
  for j in dit:
    #print(dit[j])
    if i == dit[j]:
      result.append(j)
print(result)

['agkistrodon-contortrix', 'agkistrodon-piscivorus', 'pantherophis-obsoletus', 'storeria-dekayi', 'natrix-natrix', 'lampropeltis-californiae', 'pantherophis-alleghaniensis', 'thamnophis-sirtalis', 'pantherophis-obsoletus', 'agkistrodon-piscivorus', 'agkistrodon-contortrix', 'pantherophis-alleghaniensis', 'crotalus-scutulatus', 'thamnophis-sirtalis', 'lampropeltis-californiae', 'nerodia-rhombifer', 'pituophis-catenifer', 'crotalus-atrox', 'heterodon-platirhinos', 'crotalus-atrox', 'crotalus-horridus', 'storeria-dekayi', 'crotalus-atrox', 'haldea-striatula', 'pantherophis-obsoletus', 'crotalus-atrox', 'thamnophis-radix', 'pantherophis-obsoletus', 'storeria-dekayi', 'crotalus-atrox', 'pantherophis-emoryi', 'pantherophis-obsoletus', 'thamnophis-sirtalis', 'nerodia-erythrogaster', 'thamnophis-sirtalis', 'pantherophis-alleghaniensis', 'rhinocheilus-lecontei', 'haldea-striatula', 'storeria-dekayi', 'crotalus-horridus', 'nerodia-erythrogaster', 'pantherophis-obsoletus', 'coluber-constrictor', 

In [151]:
test['breed']=result

In [153]:
#test=test.drop(['image'],axis=1)

In [154]:
filename='/content/drive/MyDrive/Snake detection/submission_100.csv'
test.to_csv(filename,index=False)
test.head()

,image_id,breed
0,7ede553357,agkistrodon-contortrix
1,3500b219e4,agkistrodon-piscivorus
2,d43a78d6d9,pantherophis-obsoletus
3,1f36f26994,storeria-dekayi
4,8cb85fc58e,natrix-natrix
